In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Reads the Tickers.csv file, and outputs a dataframe of all tickers. 
tickers = pd.read_csv("Tickers.csv")

In [3]:
# Creating an empty list, where the ticker names will be stored. 
tickers_lst = []

# A for loop to append all tickers from the DataFrame into a list
for i in range(len(tickers)):
    tickers_lst.append(tickers.iloc[i][0])

# One last ticker remaining in the column header, so also append that to the list 
tickers_lst.append(tickers.columns.values[0])

In [4]:
# Create an empty list where valid tickers will be stored 
valid_tickers = []
# For loop to check if each ticker is valid according to instructions, if true then append to the list 
for each_ticker in tickers_lst:
    stock = yf.Ticker(each_ticker)
    avg_volume= stock.history(start="2021-07-02", end = "2021-10-21").Volume.mean()
    if avg_volume >= 10000 and stock.info['market'] == 'us_market':
        valid_tickers.append(each_ticker)

- AUST: No data found, symbol may be delisted
- INVALIDTIC: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted


In [5]:
# Variables for start dates and end dates to get the history of the valid tickers 
start_date='2018-11-20' 
end_date="2021-11-20"

In [6]:
# Creates a dictionary with the tickers as keys and their closing prices as values
ticker_hist_dict = {}
for each_ticker in valid_tickers:
    ticker = yf.Ticker(each_ticker)
    close = ticker.history(start=start_date, end=end_date).Close
    ticker_hist_dict.update({each_ticker: close})

In [7]:
# Creating an empty list that will store the tickers of our final portfolio 
portfolio = []

In [8]:
# Adding the the stock with maximum standard deviation as our first stock in the portfolio
ticker_hist_df = pd.DataFrame(ticker_hist_dict)
std = pd.DataFrame(ticker_hist_df.pct_change().std())
std.columns = ["Standard deviations"]
max_std = max(ticker_hist_df.pct_change().std())
for index, row in std.iterrows():
    if max_std == row["Standard deviations"]:
        portfolio.append(index)

In [9]:
# Creating a dictionary that will store the correlation between all the remaining stocks
# and the stock with the maximum standard deviation in the portfolio that was previously chosen
ticker = yf.Ticker(portfolio[0]).history(start=start_date, end =end_date).Close
correlation = {}
ticker_df = pd.DataFrame(ticker)
for each_ticker in valid_tickers: 
    if each_ticker != portfolio[0]:
        ticker_df[each_ticker] = yf.Ticker(each_ticker).history(start=start_date, end =end_date).Close      
        correlation.update({each_ticker:ticker_df.Close.corr(ticker_df[each_ticker])})
correlation

{'ABBV': -0.041828269085318386,
 'LOW': -0.7381735452910863,
 'AMZN': -0.4664694804223396,
 'AXP': -0.4068560152527074,
 'BAC': -0.5878208355894275,
 'BMBL': 0.22925990443350588,
 'BK': -0.5062377797612438,
 'SQ': 0.6546429132215283,
 'VZ': 0.6261536784954225,
 'CMCSA': 0.5809182350167506,
 'SHOP': -0.09293116244723348,
 'COST': -0.6690192880669248,
 'CSCO': 0.023379657554691846,
 'CVS': -0.6697340344890758,
 'GM': -0.50668146607828,
 'GOOG': -0.5353764043128666,
 'JPM': -0.47046224471188075,
 'IBM': 0.6488351145184957,
 'ORCL': -0.4577393255358853,
 'OXY': -0.5398132976321177,
 'DUOL': -0.21763014228504068,
 'PEP': -0.6548437222457212,
 'SLB': -0.5419086411445654,
 'SO': 0.35852852456764644,
 'SPG': -0.6202039449471809,
 'PYPL': 0.6509945956279712,
 'AAPL': -0.2811466788970365}

In [10]:
# Choosing the remaining 9 stocks according to their correlation with the first stock
for i in range(9):
    portfolio.append(max(correlation, key=correlation.get))
    del(correlation[max(correlation, key=correlation.get)])

In [11]:
# printing out our final portfolio 
portfolio

['HOOD', 'SQ', 'PYPL', 'IBM', 'VZ', 'CMCSA', 'SO', 'BMBL', 'CSCO', 'ABBV']

In [12]:
# Defining function that returns a dataframe with tickers and their closing prices
def history(list_of_tickers, start_date, end_date):
    hist_dict = {} 
    for ticker in list_of_tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, interval="1mo").Close
        hist_dict.update({ticker: hist})

    return pd.DataFrame(hist_dict).dropna()

In [13]:
# Defining function that caclulates number of shares based on list_of_pcts that contains the weighting of each stock in list_of_tickers
def num_shares_pct(list_of_tickers, hist_df, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        stock_price = hist_df.loc[:, ticker][0]
        num_shares.append((100000*(list_of_pcts[i]/100))/(stock_price))
        i += 1
    return num_shares

In [14]:
# The weighting of each stock in our portfolio
percentages = [35, 25, 5, 5, 5, 5, 5, 5, 5, 5]

In [15]:
# extracting history of the stocks in our portolio
hist_df = history(portfolio, start_date, end_date)

In [16]:
# The date we will buy our portfolio on
buy_date = "2021-11-26"

# Creating a list with the buying price of all the stocks in our portfolio
buy_price = []
for each_ticker in portfolio:
    closing_price = yf.Ticker(each_ticker).history(start=buy_date).Close.values[0]
    buy_price.append(closing_price)

In [17]:
# The number of shares bought for each stock based on our start_date 
shares = num_shares_pct(portfolio, hist_df, percentages)
shares

[789.7111967734822,
 93.25922078915606,
 17.32141597923883,
 38.19870179436443,
 93.09702519519216,
 83.17368210878078,
 77.65330977445159,
 91.74311926605505,
 85.79192917557879,
 42.334249102587044]

In [18]:
# Defining function that calculates the portfolio based on the num of shares of each ticker in the list_of_tickers 
def portfolio_calc(list_of_tickers, hist_df, num_shares):
    value = {}
    n = 0
    for ticker in list_of_tickers:
        value.update({ticker: hist_df[ticker]*num_shares[n]})
        n += 1

    each_stock_value = pd.DataFrame(value)
    return pd.DataFrame({"portfolio": each_stock_value.sum(axis=1)})

# Creating a dataframe with the portfolio value
portfolio_df = portfolio_calc(portfolio, hist_df, shares)

# Joining history of each ticker in portfolio with portfolio_df
hist_w_portfolio= pd.concat([hist_df, portfolio_df], join='inner', axis=1)

In [19]:
hist_w_portfolio

,HOOD,SQ,PYPL,IBM,VZ,CMCSA,SO,BMBL,CSCO,ABBV,portfolio
Date,,,,,,,,,,,
2021-08-01,44.320000,268.070007,288.660004,130.894501,53.707409,60.115170,64.388756,54.500000,58.280540,118.107681,100000.000000
2021-09-01,42.080002,239.839996,260.209991,131.071228,52.740669,55.409382,61.321308,49.980000,53.748047,105.483330,93054.727548
2021-10-01,34.970001,254.500000,232.589996,118.023544,51.744648,50.951275,61.667641,52.520000,55.268753,112.132866,88038.604091
2021-11-01,25.940001,208.330002,184.889999,115.557976,49.671547,49.733742,60.460411,34.259998,54.518742,114.096046,73636.844047


In [20]:
# Calculating Beta
MarketIndex_hist = history(["^GSPC"], start_date, end_date)
index_w_portfolio = pd.concat([portfolio_df,MarketIndex_hist], join='inner', axis=1)

percent_change = index_w_portfolio.pct_change()
percent_change.drop(index=percent_change.index[0], inplace=True)

MarketVar = percent_change["^GSPC"].var()
MarketVar
Beta = percent_change.cov()/MarketVar
Beta

,portfolio,^GSPC
portfolio,0.998489,0.312746
^GSPC,0.312746,1.000000


In [21]:
# Calculating sharpe ratio assuming the risk free rate of return is 0
sharpe_ratio = portfolio_df.pct_change().mean()/portfolio_df.pct_change().std()
sharpe_ratio

portfolio   -1.611675
dtype: float64

In [22]:
# Defining function that caclulates number of shares to be bought for each stock based on the prices in list_of_prices
def num_shares_lst(list_of_tickers, list_of_prices, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        num_shares.append((100000*(list_of_pcts[i]/100))/(list_of_prices[i]))
        i += 1
    return num_shares

In [23]:
# Displaying the price we are going to buy the stocks at
buy_price  

[27.920000076293945,
 212.0800018310547,
 187.7899932861328,
 115.80999755859375,
 51.18333053588867,
 50.84822463989258,
 62.040000915527344,
 33.83000183105469,
 54.349735260009766,
 115.31341552734375]

In [24]:
# Calculating the number of shares bought
buy_shares = num_shares_lst(portfolio, buy_price, percentages)
buy_shares

[1253.5816584655913,
 117.88004424818556,
 26.62548686703223,
 43.174165490075794,
 97.68805483445641,
 98.33185003822707,
 80.59316451023138,
 147.79780459279152,
 91.99676826538237,
 43.360089345496604]

In [25]:
# Creating dictionary with FinalPortfolio data
FinalPortfolio_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                 "Ticker": portfolio,
                       "Price": buy_price,
                       "Shares": buy_shares,
                      }
# Creating Dataframe from FinalPortfolio_dict
FinalPortfolio = pd.DataFrame(FinalPortfolio_dict) 
FinalPortfolio.set_index("index", inplace=True)
FinalPortfolio['Value'] = FinalPortfolio['Price']*FinalPortfolio['Shares']
FinalPortfolio["Weight"] = percentages
print("The total value of the portfolio is: $"+ str(FinalPortfolio.Value.sum(axis=0)))

The total value of the portfolio is: $100000.0


In [26]:
print("The total weight of the portfolio is: "+ str(FinalPortfolio.Weight.sum(axis=0))+"%")

The total weight of the portfolio is: 100%


In [27]:
FinalPortfolio

,Ticker,Price,Shares,Value,Weight
index,,,,,
1,HOOD,27.920000,1253.581658,35000.0,35
2,SQ,212.080002,117.880044,25000.0,25
3,PYPL,187.789993,26.625487,5000.0,5
4,IBM,115.809998,43.174165,5000.0,5
5,VZ,51.183331,97.688055,5000.0,5
6,CMCSA,50.848225,98.331850,5000.0,5
7,SO,62.040001,80.593165,5000.0,5
8,BMBL,33.830002,147.797805,5000.0,5
9,CSCO,54.349735,91.996768,5000.0,5


In [28]:
# Creating dictionary with Stocks data
Stocks_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               "Ticker": portfolio,
               "Shares": buy_shares,}  
# Creating Dataframe from Stocks_dict 
Stocks = pd.DataFrame(Stocks_dict)
Stocks.set_index("index", inplace=True)
Stocks

,Ticker,Shares
index,,
1,HOOD,1253.581658
2,SQ,117.880044
3,PYPL,26.625487
4,IBM,43.174165
5,VZ,97.688055
6,CMCSA,98.331850
7,SO,80.593165
8,BMBL,147.797805
9,CSCO,91.996768


In [29]:
# Export final dataframe to a CSV file 
Stocks.to_csv('Stocks_Group_9.csv', index=False)

Our final portfolio will only contain 10 stocks. The more diversified a portfolio is, both risk and volatility are reduced. Since our group decided to create a risky portfolio, we chose the minimum required amount of stocks because then our portfolio will be less diverse, implying it'll be more risky and volatile. 

We first calculate the standard deviation of each stock based on the daily percentage returns. Then, the initial stock that's chosen is based on the highest standard deviation. As we know, a high standard deviation means high volatility. That means the stock price is inpredictable and can change dramatically over a short period of time in any direction, resulting in a very risky stock. 

The 9 stocks with the highest positively correlated closing prices with our initial stocks were chosen as the remaining stocks to complete our portfolio. The reason for doing this is because since all of our stocks are highly correlated, most of the stocks in the portfolio are likely to move in one direction. Choosing highly correlated stocks will increase the risk of a portfolio. 

After choosing our 10 stocks, the weights of 35%, 25%, and 5% are assigned to the stocks with the highest volatility, second highest volatility, and the remaining 8 stocks respectively. The 35% of the total weight is assigned to the stock with the highest volatility because that stock has the greatest chance to fluctuate in price. The same logic is applied to the stock with the second highest volatility, while the remaining eight stocks have 5% weight each to equal a total weight of 100%. 

The resulting Beta of the portfolio should be greater than 1, implying that the portfolio is more volatile than the broader market. The Sharpe Ratio should also be very small, meaning that the overall risk of the portfolio is high with a low expected return. 